In [1]:


# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals
from io import open

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

    
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image.astype(np.uint8),interpolation="nearest")
    plt.axis("off")

import tensorflow as tf
from tensorflow.python.framework import ops
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

reset_graph = ops.reset_default_graph

from scipy.ndimage.interpolation import shift, zoom
from datetime import datetime
import progressbar

W1019 16:06:47.609210 12224 deprecation.py:323] From C:\Users\Philip\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow_core\python\compat\v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 7 * 7])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

W1019 16:06:51.013635 12224 deprecation.py:323] From <ipython-input-2-675495a953e6>:30: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W1019 16:06:51.014635 12224 deprecation.py:323] From C:\Users\Philip\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow_core\python\layers\convolutional.py:424: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
W1019 16:06:51.046643 12224 deprecation.py:323] From <ipython-input-2-675495a953e6>:40: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.Dense instead.


In [5]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [10]:
# Shift image by dx and xy
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

print("Converting to np.array to list")
X_train_augmented = X_train.tolist()
y_train_augmented = y_train.tolist()


# augment data by shifting every image in every direction
print("Augmenting data by shifting")
for dx, dy in ((5, 0), (-5, 0), (0, 5), (0, -5), (2, 0), (-2, 0), (0, 2), (0, -2)):
    for image, label in progressbar.progressbar(zip(X_train, y_train), max_value=55000):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

# augment data by zooming every image in and out
print("Augmenting data by zooming")
for image, label in progressbar.progressbar(zip(X_train, y_train), max_value=55000):
    X_train_augmented.append(clipped_zoom(image.reshape(28, 28), 0.7).reshape(-1))
    y_train_augmented.append(label)
    X_train_augmented.append(clipped_zoom(image.reshape(28, 28), 1.3).reshape(-1))
    y_train_augmented.append(label)

# convert to numpy array for imporved speed
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

X_train = X_train_augmented
y_train = y_train_augmented

In [7]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [8]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

# acc_val_summary = tf.summary.scalar('Validation Accuracy', accuracy)
# file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 5
batch_size = 100
n_batches = len(X_train)//batch_size

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        bar = progressbar.ProgressBar(max_value=n_batches)
        i = 0
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            bar.update(i)
            i += 1
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            if not True:
                pass
                summary_str = acc_val_summary.eval(feed_dict={X: X_test, y: y_test})
                file_writer.add_summary(summary_str, epoch*n_batches+i)

        bar.finish()
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Last batch accuracy:", acc_batch, "Test accuracy:", acc_test)

        save_path = saver.save(sess, "mnist_cnn_model/my_mnist_model")
# file_writer.flush()
# file_writer.close()

100% (6050 of 6050) |####################| Elapsed Time: 0:03:51 Time:  0:03:51
0 Last batch accuracy: 0.99 Test accuracy: 0.9902
100% (6050 of 6050) |####################| Elapsed Time: 0:03:47 Time:  0:03:47
1 Last batch accuracy: 1.0 Test accuracy: 0.9925
100% (6050 of 6050) |####################| Elapsed Time: 0:03:59 Time:  0:03:59
2 Last batch accuracy: 1.0 Test accuracy: 0.9929
100% (6050 of 6050) |####################| Elapsed Time: 0:03:49 Time:  0:03:49
N/A% (0 of 6050) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--3 Last batch accuracy: 1.0 Test accuracy: 0.993
100% (6050 of 6050) |####################| Elapsed Time: 0:03:42 Time:  0:03:42
4 Last batch accuracy: 1.0 Test accuracy: 0.9932


In [61]:
with tf.Session() as sess:
    saver.restore(sess, "mnist_cnn_model/my_mnist_model")
    print(logits.eval(feed_dict={X:X_valid[0].reshape(1, -1), y:y_valid[0].reshape(-1)}))

[[-27.32937   -14.231848  -24.112865   11.604443  -23.256365   29.594835
  -11.508104  -26.737427   -2.4051402  -6.2674828]]


In [54]:
y_valid[1]

0

In [9]:
os.system("rundll32.exe powrprof.dll,SetSuspendState")

0